FAISS

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter
loader=TextLoader('speech.txt')

In [9]:
doc=loader.load()
txt_splitter=CharacterTextSplitter(chunk_size=100,chunk_overlap=30)
docs=txt_splitter.split_documents(doc)

Created a chunk of size 470, which is longer than the specified 100
Created a chunk of size 347, which is longer than the specified 100
Created a chunk of size 670, which is longer than the specified 100
Created a chunk of size 984, which is longer than the specified 100
Created a chunk of size 791, which is longer than the specified 100


In [11]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [14]:
embeddings=OllamaEmbeddings(model='llama3.2:1b')
db=FAISS.from_documents(docs,embeddings)
db

In [18]:
query="How does the speaker describe the desired outcome of the War?"
docs=db.similarity_search(query)
print(docs)
doc2=db.similarity_search_with_score(query)
print(doc2)

[Document(id='7cce24c6-d0ff-4be5-8798-806ca5c7c2dc', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'), Document(id='4ddd56f0-537e-4353-a1b0-e88260ceed6d', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all

In [19]:
retriever=db.as_retriever()
retriever.invoke(query)

[Document(id='7cce24c6-d0ff-4be5-8798-806ca5c7c2dc', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='4ddd56f0-537e-4353-a1b0-e88260ceed6d', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward al

In [20]:
#similarity search can be even done to vectors also not only text
vectorres=embeddings.embed_query(query)
similres=db.similarity_search_with_score_by_vector(vectorres)
similres

[(Document(id='7cce24c6-d0ff-4be5-8798-806ca5c7c2dc', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  np.float32(5001.797)),
 (Document(id='4ddd56f0-537e-4353-a1b0-e88260ceed6d', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be 

In [21]:
#now coming to saving and loading
db.save_local('faiss_index')

In [28]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
res3=new_db.similarity_search(query)
res3

[Document(id='7cce24c6-d0ff-4be5-8798-806ca5c7c2dc', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='4ddd56f0-537e-4353-a1b0-e88260ceed6d', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendshipâ€”exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward al

In [ ]:
type(res3)

list

In [42]:
res3[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

CHROMA

In [36]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter
loader=TextLoader('speech.txt')

In [41]:
res4=chromadb.similarity_search(query)
docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

In [43]:
chromadb=Chroma.from_documents(docs,embeddings,persist_directory="./chromadb")
chromadb

In [47]:
new_db2=Chroma(persist_directory="./chromadb",embedding_function=embeddings)
res5=new_db2.similarity_search(query)
res5[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

In [49]:
#Retreiver Option
retriever=chromadb.as_retriever()
retriever.invoke(query)[0].id

'7cce24c6-d0ff-4be5-8798-806ca5c7c2dc'